In [1]:
pip install bigml --user

Note: you may need to restart the kernel to use updated packages.


In [26]:
pip install kaggle --user

Note: you may need to restart the kernel to use updated packages.


In [2]:
import bigml.api
from bigml.api import BigML

In [3]:
import sklearn as sk
import numpy as np
import pandas as pd

In [4]:
from sklearn.model_selection import train_test_split

In [5]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split

In [6]:
from pandas import read_csv

In [7]:
api=BigML ('rodolphevdl','a47cb24c1d3a04762b4b5db6ece5e4c915efa86c',project ='project/5d9d9dd05a2139288900006f')

In [8]:
source = api.create_source('KFP.csv')

In [9]:
source_test = api.create_source('KFP_dataset_test_kaggle.csv')

In [10]:
dataset = api.create_dataset(source)

In [11]:
dataset_test = api.create_dataset(source_test)

In [12]:
train_dataset = api.create_dataset(dataset, {"name": "KFP|train 80% ", "sample_rate": 0.8})
test_dataset = api.create_dataset(dataset, {"name": "KFP|validation 20% ", "sample_rate": 0.8, "out_of_bag": True})

In [13]:
ensemble_args = {"objective_field": "SeriousDlqin2yrs"}
ensemble = api.create_ensemble(train_dataset, ensemble_args)

In [14]:
evaluation = api.create_evaluation(ensemble, test_dataset)

In [15]:
api.ok(evaluation)

True

In [17]:
prediction_args = {"name": "final_prediction_"}
batch_prediction = api.create_batch_prediction(ensemble, dataset_test, {
    "header": True,
    "all_fields": True,
    "probabilities": True})

In [18]:
api.ok(batch_prediction)

True

In [20]:
api.download_batch_prediction(batch_prediction, filename='KFPKaggle.csv')

'KFPKaggle.csv'

In [21]:
df = read_csv("KFPKaggle.csv")
df2 = df[['1 probability']]

In [22]:
df2.index = np.arange(1,len(df2)+1)
df2.index.names = ['Id']

In [23]:
df2.rename(columns = {'1 probability':'Probability'}, inplace=True)

/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:4223: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(**kwargs)


In [24]:
df2= df2.iloc[[i for i in range(0,101503)]]

In [25]:
df2.to_csv('KFPKaggle.csv')

In [27]:
import kaggle
submission_file = 'KFPKaggle.csv'

In [28]:
%env KAGGLE_USERNAME=setsin
%env KAGGLE_KEY=26cbe7b5aa9036281f4eb50023c0b353

env: KAGGLE_USERNAME=setsin
env: KAGGLE_KEY=26cbe7b5aa9036281f4eb50023c0b353


In [29]:
print(submission_file)

KFPKaggle.csv


In [30]:
!head ./kaggle-submission.csv

Id,Probability
5,0.22824
6,0.01113
7,0.0282
14,0.7185600000000001
19,0.02543
22,0.01784
34,0.01212
35,0.16508
38,0.11731


In [31]:
kaggle.api.competition_submit('KFPKaggle.csv', "Ensemble", "GiveMeSomeCredit")

100%|██████████| 1.65M/1.65M [00:42<00:00, 40.3kB/s]


Successfully submitted to Give Me Some Credit